In [1]:
from scipy import stats
import matplotlib.pyplot as plt
from ozclim import tools
import seaborn as sns
import xarray as xr 
import warnings 
import itertools

warnings.filterwarnings("ignore")

%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
region_name_dicts = {5: 'Eastern U.S.', 17: 'West-Central Europe', 35: 'Eastern Asia'}
simulation_name_dicts = {'ref': 'Present', 'refshort': 'High-warming', 'pol3.7short': 'Low-warming'}

In [3]:
ref_ds = tools.get_ensemble_ds("ref", "Emissions")
refshort_ds = tools.get_ensemble_ds("refshort", "Emissions")
polshort_ds = tools.get_ensemble_ds("pol3.7short", "Emissions")

['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w10_ref_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w13_ref_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w14_ref_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w26_ref_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w28_ref_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w10_refshort_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w13_refshort_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w14_refshort_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/OZCLIM/data/analysis_data/GCHP_CAM_c48_20240312/GCHP.w26_refshort_c48.Emissions.nc4']
['/home/eleroy/proj-dirs/O

In [4]:
def get_regional_data(ds, my_variable, my_simulation, month):
    da = ds[my_variable]
    da = tools.mask_ocean_c48(da)
    da = da.sel(time=da.time.dt.month.isin(month))
    da = da.resample(time='1Y').mean()
    da = da.stack(new_dim=['sim', 'time'])
    da['name'] = simulation_name_dicts[my_simulation]

    #EUS: 
    region_num = 5
    da_EUS = tools.crop_regionmask_ar6_c48(da, region_num)
    da_EUS = da_EUS.mean(dim=['nf', 'Ydim', 'Xdim'])
    da_EUS['region_name'] = region_name_dicts[region_num]

    #WCE: 
    region_num = 17
    da_WCE = tools.crop_regionmask_ar6_c48(da, region_num)
    da_WCE = da_WCE.mean(dim=['nf', 'Ydim', 'Xdim'])
    da_WCE['region_name'] = region_name_dicts[region_num]

    #EAS: 
    region_num = 35
    da_EAS = tools.crop_regionmask_ar6_c48(da, region_num)
    da_EAS = da_EAS.mean(dim=['nf', 'Ydim', 'Xdim'])
    da_EAS['region_name'] = region_name_dicts[region_num]

    da_all = xr.concat([da_EUS, da_WCE, da_EAS],dim='concat_dim')
    return da_all

In [5]:
def get_dataframe(my_variable, month, scale_factor):
    variable_type = "Emissions"
    ref = get_regional_data(ref_ds, my_variable, "ref", month)*scale_factor
    refshort = get_regional_data(refshort_ds, my_variable, "refshort", month)*scale_factor
    polshort = get_regional_data(polshort_ds, my_variable, "pol3.7short", month)*scale_factor
    return xr.concat([ref, polshort, refshort],dim='concat_dim').to_dataframe()

In [6]:
month = 7 
scale_factor = 1e11
EmisNO_Soil_df = get_dataframe("EmisNO_Soil", month, scale_factor)
EmisNO_Total_df = get_dataframe("EmisNO_Total", month, scale_factor)
EmisISOP_Biogenic_df = get_dataframe("EmisISOP_Biogenic", month, scale_factor)
EmisISOP_Total_df = get_dataframe("EmisISOP_Total", month, scale_factor)

In [7]:
tools.set_matplotlib_font("Arial")

fig = plt.figure(figsize=(7,9), dpi=1200, constrained_layout=True)

subfigs = fig.subfigures(2, 2)

dfs = [EmisNO_Soil_df, EmisNO_Total_df, EmisISOP_Biogenic_df, EmisISOP_Total_df]
PROPS = {
        'boxprops':{'edgecolor':'k'},
        'medianprops':{'color':'k'},
        'whiskerprops':{'color':'k'},
        'capprops':{'color':'k'}
    }
my_variables = ['EmisNO_Soil', 'EmisNO_Total', 'EmisISOP_Biogenic', 'EmisISOP_Total']
xlabels = [
            r"Soil NO emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Total NO emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Biogenic isoprene emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Total isoprene emissions (10$^{-11}$ kg/m$^2$/s)",
]

colors = ["grey", "dodgerblue", "red"]
regions = ['Eastern U.S.', 'West-Central Europe', 'Eastern Asia']
titles = ["a.", "b.", "c.", "d."]
for outerind, subfig in enumerate(subfigs.flat):

    df = dfs[outerind]
    variable = my_variables[outerind]
    xlabel = xlabels[outerind]
    subfig.suptitle(titles[outerind], x=0.1)
    axs = subfig.subplots(3, 1)

    for innerind, ax in enumerate(axs.flat):
        region = regions[innerind]
        data = df[df.region_name==region]

        a = sns.boxplot(x=variable, y="region_name", ax=ax,
                        hue="name", palette=colors, width=0.4, whis=1.2, linewidth=0.8, fliersize=0.8,
                        data=data, **PROPS)

        hatches = ['////', '', 'xxxx','////', '', 'xxxx']
        for i, bar in enumerate(a.patches):
            hatch = hatches[i]
            bar.set_hatch(hatch)
            
        p_pd = data[data.name=='Present'].dropna()
        f_pd = data[data.name=='High-warming'].dropna()
        fp_pd = data[data.name=='Low-warming'].dropna()

        statistic, f_pval_ks = stats.ks_2samp(p_pd[variable], f_pd[variable])
        statistic, fp_pval_ks = stats.ks_2samp(p_pd[variable], fp_pd[variable])

        formatted_f_pval_ks = format(f_pval_ks, ".3f")
        formatted_fp_pval_ks = format(fp_pval_ks, ".3f")

        ax.text(.02, .99, "p (KS-test)="+formatted_f_pval_ks, ha='left', va='top', transform=ax.transAxes, color='red', fontsize=8)
        ax.text(.02, .89, "p (KS-test)="+formatted_fp_pval_ks, ha='left', va='top', transform=ax.transAxes, color='dodgerblue', fontsize=8)

        ax.set_ylabel(None)
        ax.set_yticks(ax.get_yticks(), ax.get_yticklabels(), rotation=90, ha='center', va='center', fontsize=11)
        ax.legend([],[], frameon=False)
        ax.spines[['right', 'top']].set_visible(False)
        ax.tick_params(axis='both', which='major', labelsize=8)

        if innerind==2:
            ax.set_xlabel(xlabel, fontsize=11)
        else:
            ax.set_xlabel(None)

plt.savefig("/Users/emmie/Documents/OZCLIM_local/figures/FigureS8.png", format="png")

In [8]:
month = 12
scale_factor = 1e11
EmisNO_Soil_df = get_dataframe("EmisNO_Soil", month, scale_factor)
EmisNO_Total_df = get_dataframe("EmisNO_Total", month, scale_factor)
EmisISOP_Biogenic_df = get_dataframe("EmisISOP_Biogenic", month, scale_factor)
EmisISOP_Total_df = get_dataframe("EmisISOP_Total", month, scale_factor)

In [9]:
tools.set_matplotlib_font("Arial")

fig = plt.figure(figsize=(7,9), dpi=1200, constrained_layout=True)

subfigs = fig.subfigures(2, 2)

dfs = [EmisNO_Soil_df, EmisNO_Total_df, EmisISOP_Biogenic_df, EmisISOP_Total_df]
PROPS = {
        'boxprops':{'edgecolor':'k'},
        'medianprops':{'color':'k'},
        'whiskerprops':{'color':'k'},
        'capprops':{'color':'k'},
    }
my_variables = ['EmisNO_Soil', 'EmisNO_Total', 'EmisISOP_Biogenic', 'EmisISOP_Total']
xlabels = [
            r"Soil NO emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Total NO emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Biogenic isoprene emissions (10$^{-11}$ kg/m$^2$/s)",
            r"Total isoprene emissions (10$^{-11}$ kg/m$^2$/s)",
]

colors = ["grey", "dodgerblue", "red"]
regions = ['Eastern U.S.', 'West-Central Europe', 'Eastern Asia']
titles = ["a.", "b.", "c.", "d."]
for outerind, subfig in enumerate(subfigs.flat):

    df = dfs[outerind]
    variable = my_variables[outerind]
    xlabel = xlabels[outerind]
    subfig.suptitle(titles[outerind], x=0.1)
    axs = subfig.subplots(3, 1)

    for innerind, ax in enumerate(axs.flat):
        region = regions[innerind]
        data = df[df.region_name==region]

        a = sns.boxplot(x=variable, y="region_name", ax=ax,
                        hue="name", palette=colors, width=0.4, whis=1.2, linewidth=0.8, fliersize=0.8,
                        data=data, **PROPS)

        hatches = ['////', '', 'xxxx','////', '', 'xxxx']
        for i, bar in enumerate(a.patches):
            hatch = hatches[i]
            bar.set_hatch(hatch)

        p_pd = data[data.name=='Present'].dropna()
        f_pd = data[data.name=='High-warming'].dropna()
        fp_pd = data[data.name=='Low-warming'].dropna()

        statistic, f_pval_ks = stats.ks_2samp(p_pd[variable], f_pd[variable])
        statistic, fp_pval_ks = stats.ks_2samp(p_pd[variable], fp_pd[variable])

        formatted_f_pval_ks = format(f_pval_ks, ".3f")
        formatted_fp_pval_ks = format(fp_pval_ks, ".3f")

        ax.text(.02, .99, "p (KS-test)="+formatted_f_pval_ks, ha='left', va='top', transform=ax.transAxes, color='red', fontsize=8)
        ax.text(.02, .89, "p (KS-test)="+formatted_fp_pval_ks, ha='left', va='top', transform=ax.transAxes, color='dodgerblue', fontsize=8)

        ax.set_ylabel(None)
        ax.set_yticks(ax.get_yticks(), ax.get_yticklabels(), rotation=90, ha='center', va='center', fontsize=11)
        ax.legend([],[], frameon=False)
        ax.spines[['right', 'top']].set_visible(False)
        ax.tick_params(axis='both', which='major', labelsize=8)

        if innerind==2:
            ax.set_xlabel(xlabel, fontsize=11)
        else:
            ax.set_xlabel(None)

plt.savefig("/Users/emmie/Documents/OZCLIM_local/figures/FigureS10.png", format="png")